<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка данных</a></span></li><li><span><a href="#Исследование-задачи" data-toc-modified-id="Исследование-задачи-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Исследование задачи</a></span></li><li><span><a href="#Борьба-с-дисбалансом" data-toc-modified-id="Борьба-с-дисбалансом-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Борьба с дисбалансом</a></span></li><li><span><a href="#Тестирование-модели" data-toc-modified-id="Тестирование-модели-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Тестирование модели</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Вывод</a></span></li><li><span><a href="#Чек-лист-готовности-проекта" data-toc-modified-id="Чек-лист-готовности-проекта-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Чек-лист готовности проекта</a></span></li></ul></div>

# Отток клиентов

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Постройте модель с предельно большим значением *F1*-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверьте *F1*-меру на тестовой выборке самостоятельно.

Дополнительно измеряйте *AUC-ROC*, сравнивайте её значение с *F1*-мерой.

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

## Подготовка данных

In [1]:
import pandas as pd

from sklearn.metrics import f1_score, roc_auc_score

from sklearn.utils import shuffle

from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.preprocessing import OrdinalEncoder

Импорт библиотек *pandas* и *sklearn*.

In [2]:
df = pd.read_csv('/datasets/Churn.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


Загрузка данных и общая информация о датафрейме.

In [3]:
df.columns = [
    'row_number',
    'customer_id',
    'surname',
    'credit_score',
    'geography',
    'gender',
    'age',
    'tenure',
    'balance',
    'num_of_products',
    'has_cr_card',
    'is_active_member',
    'estimated_salary',
    'exited'
             ]

Приведение названий столбцов к *snake case*.

In [4]:
df.isnull().sum()

row_number            0
customer_id           0
surname               0
credit_score          0
geography             0
gender                0
age                   0
tenure              909
balance               0
num_of_products       0
has_cr_card           0
is_active_member      0
estimated_salary      0
exited                0
dtype: int64

Количество пропусков в разных столбцах.

In [5]:
med_tenure = df['tenure'].median()
df['tenure'] = df['tenure'].fillna(med_tenure)

Заполняем пропуски в столбце *tenure* медианным значением.

In [6]:
df = df.drop(['surname', 'row_number', 'customer_id'], axis=1)
df.head()

,credit_score,geography,gender,age,tenure,balance,num_of_products,has_cr_card,is_active_member,estimated_salary,exited
0,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


Удаляем малоинформативные признаки *surname*, *row_number* и *customer_id*.

In [7]:
encoder = OrdinalEncoder()
encoder.fit(df[['geography', 'gender']])
df[['geography', 'gender']] = encoder.transform(df[['geography', 'gender']]).astype(int)
df.head()

,credit_score,geography,gender,age,tenure,balance,num_of_products,has_cr_card,is_active_member,estimated_salary,exited
0,619,0,0,42,2.0,0.00,1,1,1,101348.88,1
1,608,2,0,41,1.0,83807.86,1,0,1,112542.58,0
2,502,0,0,42,8.0,159660.80,3,1,0,113931.57,1
3,699,0,0,39,1.0,0.00,2,0,0,93826.63,0
4,850,2,0,43,2.0,125510.82,1,1,1,79084.10,0


Кодируем столбцы *geography* и *gender* техникой **ordinal encoding**.

In [8]:
df_main, df_test = train_test_split(df, test_size=0.2, stratify=df['exited'], random_state=12345)
df_train, df_valid = train_test_split(df_main, test_size=0.25, random_state=12345)

features_train = df_train.drop('exited', axis=1)
target_train = df_train['exited']

features_valid = df_valid.drop('exited', axis=1)
target_valid = df_valid['exited']

Разделяем исходные данные на три выборки: обучающую, валидационную и тестовую.

## Исследование задачи

In [9]:
depths = range(1, 15)
estimators = range(10, 150, 10)

Модель - **DecisionTreeClassifier**.

In [10]:
for depth in range(1, 10):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)
    print(f'max_depth = {depth}: ', end='')
    print(f1_score(target_valid, predictions_valid))

max_depth = 1: 0.0
max_depth = 2: 0.5119760479041915
max_depth = 3: 0.5230312035661219
max_depth = 4: 0.48796147672552165
max_depth = 5: 0.509493670886076
max_depth = 6: 0.5167173252279635
max_depth = 7: 0.5128983308042488
max_depth = 8: 0.5
max_depth = 9: 0.5121212121212121


Модель - **RandomForestClassifier**.

In [11]:
for d, e in zip(depths, estimators):
    model = RandomForestClassifier(random_state=12345, max_depth=d, n_estimators=e)
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)
    print(f'n_estimators = {e}, max_depth = {d}: ', end='')
    print(f1_score(target_valid, predictions_valid))

n_estimators = 10, max_depth = 1: 0.0
n_estimators = 20, max_depth = 2: 0.12334801762114538
n_estimators = 30, max_depth = 3: 0.19491525423728814
n_estimators = 40, max_depth = 4: 0.4482173174872665
n_estimators = 50, max_depth = 5: 0.485342019543974
n_estimators = 60, max_depth = 6: 0.48445171849427165
n_estimators = 70, max_depth = 7: 0.5047923322683706
n_estimators = 80, max_depth = 8: 0.5095541401273885
n_estimators = 90, max_depth = 9: 0.5174603174603174
n_estimators = 100, max_depth = 10: 0.5221518987341772
n_estimators = 110, max_depth = 11: 0.5375000000000001
n_estimators = 120, max_depth = 12: 0.5468750000000001
n_estimators = 130, max_depth = 13: 0.5412130637636081
n_estimators = 140, max_depth = 14: 0.5457364341085271


Модели с дисбалансом классов показывают не очень хороший результат (особенно **DecisionTreeClassifier**).

## Борьба с дисбалансом

**Class_weight.**

Задаем параметр *class_weight* конструкторов моделей равным *balanced* и проверяем результаты.

Модель - **DecisionTreeClassifier**.

In [12]:
for depth in range(1, 10):
    model = DecisionTreeClassifier(random_state=12345, class_weight='balanced', max_depth=depth)
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)
    print(f'max_depth = {depth}: ', end='')
    print(f1_score(target_valid, predictions_valid))

max_depth = 1: 0.49093904448105435
max_depth = 2: 0.5060827250608273
max_depth = 3: 0.5167464114832536
max_depth = 4: 0.5641025641025641
max_depth = 5: 0.5811659192825112
max_depth = 6: 0.5606773283160866
max_depth = 7: 0.5590179414542021
max_depth = 8: 0.5559440559440559
max_depth = 9: 0.5247079964061097


Модель - **RandomForestClassifier**.

In [13]:
for d, e in zip(depths, estimators):
    model = RandomForestClassifier(random_state=12345, max_depth=d, class_weight='balanced', n_estimators=e)
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)
    print(f'n_estimators = {e}, max_depth = {d}: ', end='')
    print(f1_score(target_valid, predictions_valid))

n_estimators = 10, max_depth = 1: 0.5272895467160037
n_estimators = 20, max_depth = 2: 0.567829457364341
n_estimators = 30, max_depth = 3: 0.5722488038277512
n_estimators = 40, max_depth = 4: 0.5835806132542037
n_estimators = 50, max_depth = 5: 0.5950920245398774
n_estimators = 60, max_depth = 6: 0.6082365364308342
n_estimators = 70, max_depth = 7: 0.6013071895424836
n_estimators = 80, max_depth = 8: 0.6084070796460177
n_estimators = 90, max_depth = 9: 0.6004618937644342
n_estimators = 100, max_depth = 10: 0.597812879708384
n_estimators = 110, max_depth = 11: 0.5717948717948719
n_estimators = 120, max_depth = 12: 0.5809906291834003
n_estimators = 130, max_depth = 13: 0.5544554455445545
n_estimators = 140, max_depth = 14: 0.5665236051502145


**Upsampling.**

In [14]:
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345)
    
    return features_upsampled, target_upsampled

features_upsampled, target_upsampled = upsample(features_train, target_train, 10)

Модель - **DecisionTreeClassifier**.

In [15]:
for depth in range(1, 10):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(features_upsampled, target_upsampled)
    predictions_valid = model.predict(features_valid)
    print(f'max_depth = {depth}: ', end='')
    print(f1_score(target_valid, predictions_valid))

max_depth = 1: 0.34983498349834985
max_depth = 2: 0.5060827250608273
max_depth = 3: 0.4515771997786385
max_depth = 4: 0.4515771997786385
max_depth = 5: 0.5006535947712418
max_depth = 6: 0.5104022191400833
max_depth = 7: 0.5143658023826209
max_depth = 8: 0.5021834061135372
max_depth = 9: 0.48784082535003687


Модель - **RandomForestClassifier**.

In [16]:
for d, e in zip(depths, estimators):
    model = RandomForestClassifier(random_state=12345, max_depth=d, class_weight='balanced', n_estimators=e)
    model.fit(features_upsampled, target_upsampled)
    predictions_valid = model.predict(features_valid)
    print(f'n_estimators = {e}, max_depth = {d}: ', end='')
    print(f1_score(target_valid, predictions_valid))

n_estimators = 10, max_depth = 1: 0.5359848484848486
n_estimators = 20, max_depth = 2: 0.5597722960151802
n_estimators = 30, max_depth = 3: 0.5790441176470588
n_estimators = 40, max_depth = 4: 0.5719844357976653
n_estimators = 50, max_depth = 5: 0.6005946481665015
n_estimators = 60, max_depth = 6: 0.6060606060606061
n_estimators = 70, max_depth = 7: 0.602880658436214
n_estimators = 80, max_depth = 8: 0.6118143459915613
n_estimators = 90, max_depth = 9: 0.611111111111111
n_estimators = 100, max_depth = 10: 0.6121546961325967
n_estimators = 110, max_depth = 11: 0.6224719101123596
n_estimators = 120, max_depth = 12: 0.6105263157894737
n_estimators = 130, max_depth = 13: 0.5934065934065934
n_estimators = 140, max_depth = 14: 0.6073697585768741


Модели с балансировкой показали очень хороший результат *f1-меры*. В качестве лучшей выберем **RandomForestClassifier** с гиперпараметрами **max_depth = 8** и **n_estimators = 80**.

## Тестирование модели

In [17]:
features_test = df_test.drop('exited', axis=1)
target_test = df_test['exited']

best_model = RandomForestClassifier(random_state=12345, max_depth=8, n_estimators=80, class_weight='balanced')
best_model.fit(features_train, target_train)
predictions_test = best_model.predict(features_test)
probabilities_test = best_model.predict_proba(features_test)
probabilities_one_test = probabilities_test[:, 1]
print(f'auc-roc: {roc_auc_score(target_test, probabilities_one_test)}')
print(f'f1-мера лучшей модели: {f1_score(target_test, predictions_test)}')

auc-roc: 0.860854691363166
f1-мера лучшей модели: 0.6209223847019122


## Вывод

Был проанализирован датафрейм с информацией о поведении клиентов банка. Были переименованы названия столбцов и заполнены пропуски. Данные были разбиты на выборки - обучающая, валидационная и тестовая. Затем были исследованы две модели - **DecisionTreeClassifier** и **RandomForestClassifier** с дисбалансом классов, а затем и с балансированием с помощью параметра **class_weight='balanced'**. Лучше всего себя показала модель **RandomForestClassifier** с гиперпараметром *max_depth* равным 8 и гиперпараметром *n_estimators* равным 80. Далее данная модель была проверена на тестовой выборке - она отлично показала себя на тестовых данных и выдала результат *f1-меры* равный 0,62.